In [312]:
import googlemaps
import folium
import pandas as pd
import numpy as np
import heapq
from haversine import haversine #거리구하기
from sklearn.preprocessing import minmax_scale #정규화


##data 불러오기
total_df = pd.read_csv("all_df.csv", encoding = "EUC-KR" )
df = total_df.rename(index=str, columns={"Unnamed: 0" : "ID"})
df = df.drop(columns = ['경유여부','거리'])
print(len(df))

#중복값 검사해서 drop
a  = df[ df['총밝기'] > 120]
b = list(set(a['명칭'].values))
df  = df[ ~df['명칭'].isin(b) ]
print(len(df))

##출발지, 도착지
sp = (37.506059, 127.036863) #출발지 37.506059, 127.036863
ep = (37.509122, 127.043816) #도착지


df = df.append(df.from_dict({'ID' : -1,'point_ID':'Point_SP', '위도':sp[0], '경도':sp[1], '밝기':-100
                             , '분류':'출발지', '명칭':'출발지', '총밝기' : -100 }, orient = 'index').T)

df = df.append(df.from_dict({'ID' : len(df), 'point_ID':'Point_EP', '위도':ep[0], '경도':ep[1], '밝기':100
                             , '분류':'도착지', '명칭':'도착지', '총밝기' : 100}, orient = 'index').T)

# df = df.sort_values(by='ID').reset_index(drop=True)
# #df['ID'] = df.index.valuesd
# df = df.set_index('ID')
# df


2090
1877


C:\Users\arara\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [313]:
#총 거리
d_m = haversine(sp, ep, unit='m') # in meters
d_km = haversine(sp, ep,  unit='m')  # in kilometers

#block단위 
# 경유지가 총 3개까지만 가능하니까 3개의 반경으로 나누자. 700 이상인 경우는 api다시 받기 (추후에)
dvd = d_m / 4
print( "단위거리(dvd) : ",  dvd, ", 총거리(d_m) : " , d_m)

단위거리(dvd) :  175.38377731330854 , 총거리(d_m) :  701.5351092532342


## 함수정리
1. get_points_in_range : 중심점에서 범위 내 포함되는 점들 검색
2. get_gf : G, H 구하기. return 은 F
3. get_scaled_f : F scaling

In [314]:
# 초기화
point = sp
sel_pointID = 'Point_SP'
radius = 100 #반경
closed_list = []
closed_list_id = []

#반경 내 포함 points 검색
def get_points_in_range(df, point):
    res =df [ df.apply( lambda x : haversine(point, [x['위도'], x['경도']] ,  unit='m') , axis = 1  ) < radius ]
    return res

#가중치 구하기
#1. F구하기 
def get_f( df ):
    g = np.array(df.apply(lambda x : haversine(sp, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)
    h = np.array(df.apply(lambda x : haversine(ep, [x['위도'], x['경도']] ,  unit='m') , axis = 1 )).reshape(-1,1)  
    return g+h

#2. Scaled_F : 밝기 0~100 에 맞추기
def get_scaled_f(df):
    X_MinMax_scaled = 100 - minmax_scale(df['F'], axis=0, copy=True, feature_range =(0,100))
    return X_MinMax_scaled

#3. 밝기 이용한 가중치 추가 W(weight = 4F(거리) + 6L(밝기))
def get_w(df) : 
    w = np.array(df.apply(lambda x : (x['총밝기'] * 5 ) + ( x['scaled_F'] * 5) , axis = 1)).reshape(-1,1)
    return w

In [315]:
#1. 해당 ID Close List에 추가
heapq.heappush(closed_list_id, 'Point_SP')
heapq.heappush(closed_list, point)

while( sel_pointID != 'Point_EP' ) : 
    print("시작포인트 : " , point)
    points = pd.DataFrame()
    points = get_points_in_range(df, point)    #1. 반경 내 속하는 점들 검색
    points = points[ ~points['point_ID'].isin(closed_list_id) ] # 1-2 . 이미 선택했던 point들 제외
    points['F'] = get_f(points)                #2. F=G+H : F구하기
    points['scaled_F'] = get_scaled_f(points)  #3. F를 밝기에 맞춰 스케일링
    points['W'] = get_w(points)                #4. 최종가중치 구하기
    points = points.sort_values('W', ascending = False).reset_index() #5. 정렬

    print(points.loc[0,'point_ID'])
    sel_pointID = points.loc[0,'point_ID']
    point = (points.loc[0,'위도'] , points.loc[0,'경도'])

    heapq.heappush(closed_list_id, sel_pointID)
    heapq.heappush(closed_list, ( points.loc[0,'위도'] , points.loc[0,'경도']))
   # print(sel_pointID)

시작포인트 :  (37.506059, 127.036863)
Point_1380
시작포인트 :  (37.50617371, 127.03788814200001)
Point_1379
시작포인트 :  (37.5064151619, 127.037942552)
Point_1805
시작포인트 :  (37.5068128, 127.037784)
Point_1382
시작포인트 :  (37.50617371, 127.03788814200001)
Point_1381
시작포인트 :  (37.50617371, 127.03788814200001)
Point_1321
시작포인트 :  (37.5067072649, 127.037386249)
Point_1503
시작포인트 :  (37.5060619679, 127.03794237299999)
Point_1373
시작포인트 :  (37.5067287276, 127.037892947)
Point_1372
시작포인트 :  (37.5067287276, 127.037892947)
Point_1371
시작포인트 :  (37.5068512947, 127.037798005)
Point_1834
시작포인트 :  (37.506710999999996, 127.037181)
Point_1326
시작포인트 :  (37.5066064664, 127.03702428)
Point_1322
시작포인트 :  (37.5067540684, 127.03754008799999)
Point_2072
시작포인트 :  (37.506303, 127.03824099999999)
Point_1504
시작포인트 :  (37.5063931751, 127.039055437)
Point_1445
시작포인트 :  (37.5063931751, 127.039055437)
Point_703
시작포인트 :  (37.5061158246, 127.03857123)
Point_1376
시작포인트 :  (37.5061158246, 127.03857123)
Point_2023
시작포인트 :  (37.5066090000000

In [ ]:
closed_list

In [316]:
# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map1 = folium.Map(location= sp, zoom_start=18)
a = closed_list

# for n in range(0, len(a)) :
#     if a['분류'][n] == '편의점' :
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='green')).add_to(to_map1)
#     elif a['분류'][n] == 'CCTV' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)        
#     elif a['분류'][n] == '보안등' : 
#         folium.CircleMarker([a['위도'][n], a['경도'][n]], popup= a['point_ID'][n] + a['명칭'][n], radius = a['밝기'][n] + 1).add_to(to_map1)  
#     else : 
#         folium.Marker([a['위도'][n], a['경도'][n]], popup= a['분류'][n], icon=folium.Icon(color='black')).add_to(to_map1)        

        
for n in range(0, len(a)) :
      folium.CircleMarker([a[n][0], a[n][1]], popup= closed_list_id[n], radius = 3).add_to(to_map1)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red', )).add_to(to_map1)
to_map1


In [ ]:
df[df['point_ID'].isin(closed_list_id) ]

In [318]:
dd = total_df[ total_df['point_ID'].isin(closed_list_id) ].sort_values('총밝기', ascending = False)
dd

,Unnamed: 0,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
1376,1376,Point_1376,0,20,120,보안등,역삼1동 661-15,37.506116,127.038571,0
703,703,Point_703,0,20,120,보안등,역삼1동 661-15,37.506116,127.038571,0
1805,1805,Point_1805,0,20,120,편의점,GS25 강남갤러리점,37.506813,127.037784,0
1387,1387,Point_1387,0,20,120,보안등,역삼1동 657-23,37.505983,127.036558,0
1615,1615,Point_1615,0,20,120,보안등,역삼1동 665,37.505086,127.036539,0
1614,1614,Point_1614,0,20,120,보안등,역삼1동 665,37.505643,127.037918,0
1504,1504,Point_1504,0,20,120,보안등,역삼1동 661-26,37.506393,127.039055,0
1503,1503,Point_1503,0,20,120,보안등,역삼1동 661-1,37.506062,127.037942,0
1498,1498,Point_1498,0,20,120,보안등,역삼1동 660-12,37.505316,127.036517,0
1321,1321,Point_1321,0,20,120,보안등,역삼1동 656,37.506707,127.037386,0


In [279]:
total_df[ total_df['point_ID'] == 'Point_247']

,Unnamed: 0,point_ID,거리,밝기,총밝기,분류,명칭,위도,경도,경유여부
247,247,Point_247,0,20,100,보안등,역삼1동 684-5,37.508331,127.042024,0


In [308]:


res =df [ df.apply( lambda x : haversine([37.508331, 127.042024], [x['위도'], x['경도']] ,  unit='m') , axis = 1  ) < 150 ]

In [309]:
res = res.reset_index(drop=True)

In [310]:
# 역삼동 중심 = 역삼역 /  좌표 : 37.500742, 127.036891
to_map2 = folium.Map(location= sp, zoom_start=18)

        
for n in range(0, len(res)) :
      folium.CircleMarker([res.loc[n,'위도'], res.loc[n,'경도']] , radius = 2).add_to(to_map2)        

# 구분하기 쉽게 스타팅 지역은 빨간색 마커로 표시        
folium.Marker(sp, popup= 'SP', icon=folium.Icon(color='red', )).add_to(to_map2)
to_map2

In [311]:
res.loc[,'위도']

SyntaxError: invalid syntax (<ipython-input-311-ea816273aecb>, line 1)